# Co-Expression Analysis

--> Add a description HERE!

## Import libraries

In [ ]:
import PyWGCNA

import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql import SparkSession

## Setup spark session

In [ ]:
spark = (
    SparkSession.builder
    .config("spark.driver.memory", "20g")
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .appName("single-cell-reciter-job")
    .master("local[*]")
    .getOrCreate()
)

In [ ]:
spark

## Load the data

In [ ]:
data = spark.read.parquet('./Results/parquets/rank_by_topic')

In [ ]:
data.show()

In [ ]:
data.count()

In [ ]:
topic_list = []
col_names = [x[0] for x in data.select('topic_0').toLocalIterator()]

schema = T.StructType(
    [T.StructField(name, T.DoubleType(), False) for name in col_names]
    )

df_base = spark.createDataFrame([], schema=schema)

In [ ]:
for i in range(10):
    topic_list.append(f'topic_{i}')
    col_list = [col for col in data.columns if f'{i}' in col]
    #col_names = [x[0] for x in data.select(col_list[0]).toLocalIterator()]
    #values = [x[0] for x in data.select(col_list[1]).toLocalIterator()]
    df_base = df_base.unionByName(spark.createDataFrame([[x[0] for x in data.select(col_list[1]).toLocalIterator()],], [x[0] for x in data.select(col_list[0]).toLocalIterator()]))

In [ ]:
df_base.write.parquet("../results/base.parquet")

In [ ]:
df_base.count()

In [ ]:
len(df_base.columns)

## Analysis

### Load the data

In [ ]:
data = spark.read.parquet("../results/base.parquet")

In [ ]:
pd_df= data.toPandas()

In [ ]:
transposed_data = pd_df.T

In [ ]:
transposed_data = transposed_data.reset_index()

### Create a PyWGCNA object

In [ ]:
# Not necessary
expression_matrix = PyWGCNA.geneExp.GeneExp(species='human', level='transcript', geneExp=transposed_data)

In [ ]:
# create a PyWGCNA object
scc = PyWGCNA.WGCNA(name='SCC', species='human', geneExp=transposed_data, save=True, TPMcutoff=1)

In [ ]:
scc.geneExpr.to_df().head(5)

### Pre-processing

In [ ]:
scc.preprocess()

With TPMcutoff = 1 (default):

Output:
Pre-processing...
	Detecting genes and samples with too many missing values...
An exception has occurred, use %tb to see the full traceback.

SystemExit: Too few genes with valid expression levels in the required number of samples.

With TPMcutoff = 0 : 

ValueError: Image size of 138900x1000 pixels is too large. It must be less than 2^16 in each direction.

--> Set save=False

--> Same error...

### Gene network

In [ ]:
scc.findModules()